In [255]:
import pandas as pd
import numpy as np

In [256]:
sanrenpuku = pd.read_csv("ODDS_SANREN.txt", header=None)
umatan = pd.read_csv("ODDS_UMATAN.txt", header=None)

In [257]:
harai = pd.read_csv("HARAI.txt", header=None)

In [340]:
harai[156]

0          2
1          1
2         17
3         16
4         82
5          4
6         28
7         47
8        139
9          4
10         5
11         1
12         6
13        13
14        32
15        12
16         5
17        21
18       180
19         7
20         8
21         7
22        21
23         1
24         7
25         6
26        35
27        68
28        25
29        50
        ... 
15896      6
15897      8
15898     32
15899     39
15900      2
15901      1
15902     22
15903      5
15904     61
15905     33
15906      6
15907      8
15908     17
15909     47
15910     17
15911     14
15912     50
15913     16
15914      7
15915    118
15916     37
15917     23
15918      2
15919      4
15920     40
15921     49
15922      8
15923     63
15924      8
15925     62
Name: 156, Length: 15926, dtype: int64

In [341]:
#harai[173][174]
harai_sanrenpuku = pd.concat([harai[173], harai[174]], axis=1)
harai_umatan = pd.concat([harai[155], harai[156]], axis=1)

In [342]:
harai_umatan

,155,156
0,1330,2
1,620,1
2,4090,17
3,3750,16
4,43850,82
5,1440,4
6,6150,28
7,13930,47
8,67460,139
9,1350,4


In [343]:
harai_sanrenpuku

,173,174
0,7070,22
1,1600,4
2,4160,11
3,3050,9
4,11910,37
5,1080,2
6,5940,21
7,6140,14
8,68980,162
9,1050,3


In [258]:
sanrenpuku.head()

,0,1,2,3,4,5,6,7,8,9
0,20150106,2015,104,6,1,1,1,10203,019224,211
1,20150106,2015,104,6,1,1,1,10204,023246,234
2,20150106,2015,104,6,1,1,1,10205,010141,153
3,20150106,2015,104,6,1,1,1,10206,091027,393
4,20150106,2015,104,6,1,1,1,10207,087851,387


In [259]:
umatan.head()

,0,1,2,3,4,5,6,7,8,9
0,20150106,2015,104,6,1,1,1,102,020602,154
1,20150106,2015,104,6,1,1,1,103,013444,132
2,20150106,2015,104,6,1,1,1,104,011412,126
3,20150106,2015,104,6,1,1,1,105,005585,091
4,20150106,2015,104,6,1,1,1,106,032048,175


In [260]:
sanrenpuku.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6850205 entries, 0 to 6850204
Data columns (total 10 columns):
0    int64
1    int64
2    int64
3    int64
4    int64
5    int64
6    int64
7    int64
8    object
9    object
dtypes: int64(8), object(2)
memory usage: 522.6+ MB


In [261]:
umatan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3126004 entries, 0 to 3126003
Data columns (total 10 columns):
0    int64
1    int64
2    int64
3    int64
4    int64
5    int64
6    int64
7    int64
8    object
9    object
dtypes: int64(8), object(2)
memory usage: 238.5+ MB


In [262]:
harai.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15926 entries, 0 to 15925
Columns: 199 entries, 0 to 198
dtypes: float64(8), int64(110), object(81)
memory usage: 24.2+ MB


In [263]:
umatan[umatan[8] == '------']

,0,1,2,3,4,5,6,7,8,9
7342,20150106,2015,105,8,1,2,1,106,------,---
7357,20150106,2015,105,8,1,2,1,206,------,---
7372,20150106,2015,105,8,1,2,1,306,------,---
7387,20150106,2015,105,8,1,2,1,406,------,---
7402,20150106,2015,105,8,1,2,1,506,------,---
7413,20150106,2015,105,8,1,2,1,601,------,---
7414,20150106,2015,105,8,1,2,1,602,------,---
7415,20150106,2015,105,8,1,2,1,603,------,---
7416,20150106,2015,105,8,1,2,1,604,------,---
7417,20150106,2015,105,8,1,2,1,605,------,---


In [264]:
umatan[umatan[8] == '******']

,0,1,2,3,4,5,6,7,8,9
13047,20150113,2015,110,8,1,3,5,115,******,***
13062,20150113,2015,110,8,1,3,5,215,******,***
13077,20150113,2015,110,8,1,3,5,315,******,***
13092,20150113,2015,110,8,1,3,5,415,******,***
13107,20150113,2015,110,8,1,3,5,515,******,***
13122,20150113,2015,110,8,1,3,5,615,******,***
13137,20150113,2015,110,8,1,3,5,715,******,***
13152,20150113,2015,110,8,1,3,5,815,******,***
13167,20150113,2015,110,8,1,3,5,915,******,***
13182,20150113,2015,110,8,1,3,5,1015,******,***


In [265]:
sanrenpuku[sanrenpuku[8] == '------']

,0,1,2,3,4,5,6,7,8,9
16330,20150106,2015,105,8,1,2,1,10206,------,---
16343,20150106,2015,105,8,1,2,1,10306,------,---
16355,20150106,2015,105,8,1,2,1,10406,------,---
16366,20150106,2015,105,8,1,2,1,10506,------,---
16377,20150106,2015,105,8,1,2,1,10607,------,---
16378,20150106,2015,105,8,1,2,1,10608,------,---
16379,20150106,2015,105,8,1,2,1,10609,------,---
16380,20150106,2015,105,8,1,2,1,10610,------,---
16381,20150106,2015,105,8,1,2,1,10611,------,---
16382,20150106,2015,105,8,1,2,1,10612,------,---


In [266]:
sanrenpuku[sanrenpuku[8] == '******']

,0,1,2,3,4,5,6,7,8,9
28850,20150113,2015,110,8,1,3,5,10215,******,***
28863,20150113,2015,110,8,1,3,5,10315,******,***
28875,20150113,2015,110,8,1,3,5,10415,******,***
28886,20150113,2015,110,8,1,3,5,10515,******,***
28896,20150113,2015,110,8,1,3,5,10615,******,***
28905,20150113,2015,110,8,1,3,5,10715,******,***
28913,20150113,2015,110,8,1,3,5,10815,******,***
28920,20150113,2015,110,8,1,3,5,10915,******,***
28926,20150113,2015,110,8,1,3,5,11015,******,***
28931,20150113,2015,110,8,1,3,5,11115,******,***


In [267]:
sanrenpuku.drop(sanrenpuku.index[sanrenpuku[8] == '******'], inplace=True)
sanrenpuku.drop(sanrenpuku.index[sanrenpuku[8] == '------'], inplace=True)

In [268]:
umatan.drop(umatan.index[umatan[8] == '******'], inplace=True)
umatan.drop(umatan.index[umatan[8] == '------'], inplace=True)

In [269]:
sanrenpuku = sanrenpuku.astype('int64')
umatan = umatan.astype('int64')

In [270]:
sanrenpuku.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6768795 entries, 0 to 6850204
Data columns (total 10 columns):
0    int64
1    int64
2    int64
3    int64
4    int64
5    int64
6    int64
7    int64
8    int64
9    int64
dtypes: int64(10)
memory usage: 568.1 MB


In [271]:
umatan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3099952 entries, 0 to 3126003
Data columns (total 10 columns):
0    int64
1    int64
2    int64
3    int64
4    int64
5    int64
6    int64
7    int64
8    int64
9    int64
dtypes: int64(10)
memory usage: 260.2 MB


ほしいのは，0~6が同じでrankをカラムとした8(harai)の値

In [272]:
import gc
gc.collect()

241

In [286]:
sanrenpuku['key'] = sanrenpuku[1].astype(str) + sanrenpuku[2].astype(str) + sanrenpuku[3].astype(str) + sanrenpuku[4].astype(str) + sanrenpuku[5].astype(str) + sanrenpuku[6].astype(str)
umatan['key'] = umatan[1].astype(str) + umatan[2].astype(str) + umatan[3].astype(str) + umatan[4].astype(str) + umatan[5].astype(str) + umatan[6].astype(str)

In [287]:
sanrenpuku['key']

196           20151046111
279           20151046111
213           20151046111
278           20151046111
374           20151046111
281           20151046111
212           20151046111
381           20151046111
215           20151046111
280           20151046111
380           20151046111
214           20151046111
376           20151046111
385           20151046111
202           20151046111
384           20151046111
375           20151046111
284           20151046111
319           20151046111
218           20151046111
201           20151046111
106           20151046111
326           20151046111
253           20151046111
204           20151046111
118           20151046111
526           20151046111
260           20151046111
135           20151046111
321           20151046111
                ...      
5589091    20189021021212
5588930    20189021021212
5589169    20189021021212
5589431    20189021021212
5589135    20189021021212
5588927    20189021021212
5588993    20189021021212
5588963    2

In [288]:
umatan['key']

0            20151046111
1            20151046111
2            20151046111
3            20151046111
4            20151046111
5            20151046111
6            20151046111
7            20151046111
8            20151046111
9            20151046111
10           20151046111
11           20151046111
12           20151046111
13           20151046111
14           20151046111
15           20151046111
16           20151046111
17           20151046111
18           20151046111
19           20151046111
20           20151046111
21           20151046111
22           20151046111
23           20151046111
24           20151046111
25           20151046111
26           20151046111
27           20151046111
28           20151046111
29           20151046111
               ...      
3125974    2019804102412
3125975    2019804102412
3125976    2019804102412
3125977    2019804102412
3125978    2019804102412
3125979    2019804102412
3125980    2019804102412
3125981    2019804102412
3125982    2019804102412


In [289]:
sanrenpuku.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6768795 entries, 196 to 5589301
Data columns (total 11 columns):
0      int64
1      int64
2      int64
3      int64
4      int64
5      int64
6      int64
7      int64
8      int64
9      int64
key    object
dtypes: int64(10), object(1)
memory usage: 939.7+ MB


In [290]:
sanrenpuku = sanrenpuku.astype('int64')
umatan = umatan.astype('int64')

In [301]:
sanrenpuku.head()

,0,1,2,3,4,5,6,7,8,9,key
196,20150106,2015,104,6,1,1,1,30405,145,1,20151046111
24,20150106,2015,104,6,1,1,1,10314,85855,383,20151046111
87,20150106,2015,104,6,1,1,1,11014,84890,382,20151046111
141,20150106,2015,104,6,1,1,1,20607,83947,379,20151046111
116,20150106,2015,104,6,1,1,1,20315,83947,379,20151046111


In [302]:
umatan.head()

,0,1,2,3,4,5,6,7,8,9,key
0,20150106,2015,104,6,1,1,1,102,20602,154,20151046111
152,20150106,2015,104,6,1,1,1,1103,539,19,20151046111
153,20150106,2015,104,6,1,1,1,1104,437,15,20151046111
154,20150106,2015,104,6,1,1,1,1105,212,6,20151046111
155,20150106,2015,104,6,1,1,1,1106,5696,92,20151046111


In [304]:
sanrenpuku.rename(columns={8:'harai',9: 'rank'}, inplace=True)
umatan.rename(columns={8:'harai',9: 'rank'}, inplace=True)

# ソート

In [306]:
sanrenpuku.sort_values(['key','rank'], inplace=True)
umatan.sort_values(['key', 'rank'], inplace=True)

In [314]:
sanrenpuku.reset_index(drop=True, inplace=True)
umatan.reset_index(drop=True, inplace=True)

In [315]:
sanrenpuku

,0,1,2,3,4,5,6,7,harai,rank,key
0,20150106,2015,104,6,1,1,1,30405,145,1,20151046111
1,20150106,2015,104,6,1,1,1,40511,148,2,20151046111
2,20150106,2015,104,6,1,1,1,30511,185,3,20151046111
3,20150106,2015,104,6,1,1,1,40510,205,4,20151046111
4,20150106,2015,104,6,1,1,1,51011,222,5,20151046111
5,20150106,2015,104,6,1,1,1,40513,252,6,20151046111
6,20150106,2015,104,6,1,1,1,30510,258,7,20151046111
7,20150106,2015,104,6,1,1,1,51113,259,8,20151046111
8,20150106,2015,104,6,1,1,1,30513,262,9,20151046111
9,20150106,2015,104,6,1,1,1,40512,319,10,20151046111


In [316]:
umatan

,0,1,2,3,4,5,6,7,harai,rank,key
0,20150106,2015,104,6,1,1,1,504,132,1,20151046111
1,20150106,2015,104,6,1,1,1,503,133,2,20151046111
2,20150106,2015,104,6,1,1,1,511,136,3,20151046111
3,20150106,2015,104,6,1,1,1,405,176,4,20151046111
4,20150106,2015,104,6,1,1,1,510,182,5,20151046111
5,20150106,2015,104,6,1,1,1,1105,212,6,20151046111
6,20150106,2015,104,6,1,1,1,513,214,7,20151046111
7,20150106,2015,104,6,1,1,1,305,225,8,20151046111
8,20150106,2015,104,6,1,1,1,512,253,9,20151046111
9,20150106,2015,104,6,1,1,1,1005,265,10,20151046111


In [318]:
sanrenpuku.describe()

,0,1,2,3,4,5,6,7,harai,rank,key
count,6.768795e+06,6.768795e+06,6.768795e+06,6.768795e+06,6.768795e+06,6.768795e+06,6.768795e+06,6.768795e+06,6.768795e+06,6.768795e+06,6.768795e+06
mean,2.016842e+07,2.016779e+03,6.244287e+02,6.265737e+00,2.490998e+00,4.776018e+00,6.383100e+00,4.204908e+04,6.371985e+04,2.616945e+02,3.161868e+11
std,1.326917e+04,1.331462e+00,3.424230e+02,2.315826e+00,1.280111e+00,2.695377e+00,3.495391e+00,2.859887e+04,1.162920e+05,1.810357e+02,1.092682e+12
min,2.015011e+07,2.015000e+03,1.040000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.020300e+04,0.000000e+00,1.000000e+00,2.015105e+10
25%,2.016022e+07,2.016000e+03,3.210000e+02,5.000000e+00,1.000000e+00,3.000000e+00,3.000000e+00,2.050700e+04,5.037000e+03,1.100000e+02,2.017326e+10
50%,2.017032e+07,2.017000e+03,6.090000e+02,6.000000e+00,2.000000e+00,5.000000e+00,6.000000e+00,3.131400e+04,2.029700e+04,2.350000e+02,2.019714e+10
75%,2.018051e+07,2.018000e+03,9.120000e+02,8.000000e+00,3.000000e+00,7.000000e+00,9.000000e+00,6.081300e+04,6.937800e+04,3.900000e+02,2.017313e+11
max,2.019080e+07,2.019000e+03,1.228000e+03,1.000000e+01,5.000000e+00,1.200000e+01,1.200000e+01,1.617180e+05,9.999990e+05,8.160000e+02,2.018902e+13


In [320]:
sanrenpuku.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6768795 entries, 0 to 6768794
Data columns (total 11 columns):
0        int64
1        int64
2        int64
3        int64
4        int64
5        int64
6        int64
7        int64
harai    int64
rank     int64
key      int64
dtypes: int64(11)
memory usage: 568.1 MB


In [323]:
odds_sanrenpuku = pd.pivot_table(sanrenpuku, values="harai",index="key", columns="rank")
odds_umatan = pd.pivot_table(umatan, values="harai",index="key", columns="rank")

In [328]:
odds_sanrenpuku.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15926 entries, 20151046111 to 20189021021212
Columns: 816 entries, 1 to 816
dtypes: float64(816)
memory usage: 99.3 MB


In [329]:
odds_umatan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15926 entries, 20151046111 to 20189021021212
Columns: 306 entries, 1 to 306
dtypes: float64(306)
memory usage: 37.3 MB


In [344]:
odds_sanrenpuku.to_csv('odds_sanrenpuku.csv')
odds_umatan.to_csv('odds_umatan.csv')
harai_sanrenpuku.to_csv('harai_sanrenpuku.csv')
harai_umatan.to_csv('harai_umatan.csv')